In [13]:
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

import pandas as pd

In [14]:
#Folder for the dataset
datasetFolder = '/home/carnd/dbpedia2016/dataset/'

#Number of files
numberOfFiles = 511

#Test split
testSplit=0.25

In [15]:
def load_data(datasetFolder, datasetXFile, datasetYFile):
    print('Loading {} & {}'.format(datasetXFile, datasetYFile))
    # load file
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        cols = head.split(',')
        numberOfCols = len(cols)
        #print(numberOfCols)
        numberOfRows=0
        for line in f:
            numberOfRows+=1
        f.close()
    
    #print('{} x {}'.format(numberOfRows,numberOfCols))
    dataX = np.zeros([numberOfRows,numberOfCols-1],np.float32)
    
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        rowCounter=0
        for line in f:
            row=line.split(',')
            for i in range(1, len(row)):
                dataX[rowCounter][int(row[i])-1]=1.0
            rowCounter+=1
        f.close()
        
    dataY=pd.read_csv(datasetFolder + datasetYFile)

    # delete the id fields
    #del dataX['id']
    del dataY['id']

    return dataX, dataY.as_matrix()

In [16]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv')

Loading datasetX_1.csv & datasetY_1.csv


In [17]:
print(dataX.shape)
print(dataX[0:5])

(5989, 5171)
[[ 1.  0.  0. ...,  0.  1.  0.]
 [ 0.  1.  1. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 1.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]


In [18]:
print(dataY.shape)
print(dataY[0:5])

(5989, 539)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [19]:
print("Input Features: {}".format(dataX.shape[1]))
print("Output Classes: {}".format(dataY.shape[1]))

Input Features: 5171
Output Classes: 539


In [20]:
# create model
logisticRegression = Sequential(name='Simple Logistic Regression')
logisticRegression.add(Dense(dataY.shape[1], input_dim=dataX.shape[1], activation='sigmoid', init='glorot_uniform'))

simpleModel = Sequential(name='2 Fully Connected Layers')
simpleModel.add(Dense(1024, input_dim=dataX.shape[1], activation='relu', init='glorot_uniform'))
simpleModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_uniform'))

deepModel = Sequential(name='Deep Model (5 Dense Layers)')
deepModel.add(Dense(2048, input_dim=dataX.shape[1], activation='relu', init='glorot_uniform'))
deepModel.add(Dense(1024, activation='relu', init='glorot_uniform'))
deepModel.add(Dense(768, activation='relu', init='glorot_uniform'))
deepModel.add(Dense(512, activation='relu', init='glorot_uniform'))
deepModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_uniform'))

models = [logisticRegression, simpleModel, deepModel]

In [21]:
# Compile model
import keras.backend as K

def count_predictions(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives, predicted_positives, possible_positives

def f1score(y_true, y_pred):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = 2.0 * precision * recall / (precision+recall+ K.epsilon())
    return f1score

def fBetaScore(y_true, y_pred, beta):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = (1+(beta*beta)) * precision * recall / ((beta*beta*precision)+recall+ K.epsilon())
    return f1score

for model in models:
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1score])

In [22]:
def fit_data(model, dataX, dataY):
    # Fit the model
    #model.fit(dataX, dataY, nb_epoch=5, verbose=2, batch_size=256)
    return model.train_on_batch(dataX, dataY)

In [23]:
def countPredictions(y_true, y_pred):
    true_positives = np.sum(np.round(y_pred*y_true))
    predicted_positives = np.sum(np.round(y_pred))
    possible_positives = np.sum(y_true)
    return true_positives, predicted_positives, possible_positives

In [24]:
#Randomize the list of numbers so we can split train and test dataset
listOfFiles=list(range(1,numberOfFiles+1))
import random
random.shuffle(listOfFiles)
splitIndex=int((1-testSplit)*numberOfFiles)

numberOfEons = 5
for eon in range(0, numberOfEons):
    print('\n***********************************\n {}/{}'.format(eon+1, numberOfEons))
    for trainIndex in range(0,splitIndex):
        print('==========================================')
        print('Learning for file {} / {} : datasetX/Y_{}'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex]))
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[trainIndex]), 'datasetY_{}.csv'.format(listOfFiles[trainIndex]))
        print('Training...')
        for model in models:
            #print('Model = {}'.format(model.name))
            loss, f1score=fit_data(model,dataX, dataY)
            print('   Model = {} \t loss = {:.4f} \t f1-score = {:.4f}'.format(model.name, loss, f1score))
            
        
    counts = {} 
    for model in models:
        counts[model.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
    
    print('==========================================\n')
    for testIndex in range(splitIndex, numberOfFiles):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
        print ('Testing for file {} / {} : datasetX/Y_{}'.format(testIndex+1, numberOfFiles, listOfFiles[trainIndex]))
        for model in models:
            predY=model.predict_on_batch(dataX)
            true_positives, predicted_positives, possible_positives = countPredictions(dataY, predY)
            counts[model.name]['true_positives'] += true_positives
            counts[model.name]['predicted_positives'] += predicted_positives
            counts[model.name]['possible_positives'] += possible_positives
    print('==========================================\n')
    
    
    print('{}.  {}/{} - Testing score:'.format(eon+1, eon+1, numberOfEons))
    for model in models:
        count = counts[model.name]
        precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
        recall = (count['true_positives'])/(count['possible_positives']+0.0001)
        f1score = 2.0 * precision * recall / (precision+recall+0.0001)
        print(' - Model = {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(model.name, f1score, precision, recall))
    print('==========================================\n')


***********************************
Eon 1/5
Learning for file 1 / 383 : datasetX/Y_29
Loading datasetX_29.csv & datasetY_29.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.6933 	 f1-score = 0.0124
   Model = 2 Fully Connected Layers 	 loss = 0.6930 	 f1-score = 0.0128
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6935 	 f1-score = 0.0124
Learning for file 2 / 383 : datasetX/Y_359
Loading datasetX_359.csv & datasetY_359.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.6918 	 f1-score = 0.0129
   Model = 2 Fully Connected Layers 	 loss = 0.6866 	 f1-score = 0.0131
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6879 	 f1-score = 0.0131
Learning for file 3 / 383 : datasetX/Y_146
Loading datasetX_146.csv & datasetY_146.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.6900 	 f1-score = 0.0135
   Model = 2 Fully Connected Layers 	 loss = 0.6788 	 f1-score = 0.0131
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6729 	 f1-score = 0.016

==================================================
# length = 1, number of walks = 5 
==================================================
1. Eon 1/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.4395	 precision = 0.9003 	 recall = 0.2907
 - Model = 2 Fully Connected Layers 	 f1-score = 0.6796	 precision = 0.8532 	 recall = 0.5647
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.7986	 precision = 0.8785 	 recall = 0.7322
2. Eon 2/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.5169	 precision = 0.8813 	 recall = 0.3658
 - Model = 2 Fully Connected Layers 	 f1-score = 0.7648	 precision = 0.8728 	 recall = 0.6806
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8118	 precision = 0.8762 	 recall = 0.7564
3. Eon 3/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.5821	 precision = 0.8779 	 recall = 0.4355
 - Model = 2 Fully Connected Layers 	 f1-score = 0.7904	 precision = 0.8729 	 recall = 0.7222
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8144	 precision = 0.8770 	 recall = 0.7602
4. Eon 4/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.6311	 precision = 0.8722 	 recall = 0.4946
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8001	 precision = 0.8744 	 recall = 0.7375
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8161	 precision = 0.8746 	 recall = 0.7650
5. Eon 5/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.6567	 precision = 0.8705 	 recall = 0.5273
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8049	 precision = 0.8757 	 recall = 0.7448
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8169	 precision = 0.8743 	 recall = 0.7666